# <center> Lesson 4a - Cryptographic Primitives - Symmetric Encryption
## <center> SYSE 541: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 
### <center> Instructor: Dr. Jeremy Daily

## Objective
In this lesson, we will introduce a protocol for authentication. This protocol will depend on a pre-shared key and symmetric encryption. Therefore, we'll discuss implementing symmetric encryption recipes and discuss the challenges and issues associated with symmetric encryption algorithms.

## Challenge-Response Authentication

Let's say there is a remote process controller that you would like to interface with. The controller will act like a server and listen for connections. We'll denote the server as S and the user as the client, or C. 


The protocol could follow these steps:
1. The Client requests a seed or nonce.
2. The Server sends a nonce (number used once).
3. The Client combines its unique ID with the nonce, encrypts it with a symmetric key, and sends it back to the Server.
4. The Server decrypts the message, verifies the nonce and checks the client's unique ID against a master list. 

In symbol form, it might look like this:

$ S\rightarrow C: N $

$ C\rightarrow S: C,\{C,N\}_K $

Where $C$ and $S$ represent the client and server when it's used on the left side of the colon. On the right side of the colon $C$ is the unique ID value for the Client, $N$ is a nonce. $K$ is the symmetric encryption key, so everything inside the curly braces is encrypted with K. Commas represent concatenation.

See Secion 4.3.1 in Anderson for an introduction or review for this protocol.
Let's see if we can implement this.

### Number used once
A nonce in the protocol can come from many sources. It depends on the resources and time constraints. Perhaps the best nonce is a universally unique identifier (UUID), which combines randomness and freshness. Another is a real-time clock counter. Very simple nonces are program counters. However, a program counter is easy to reset and get low number of options. The real-time may not be available to an embedded device. We'll use a 32-byte random number as our nonce.

In [1]:
# Use the operating system for random number generation
from os import urandom
number_of_bytes = 32
nonce = urandom(number_of_bytes)
nonce

b'\xf7J\xcdY\r\xbbf%\xf7b\x12\x01\xf8[Y\xf43\xb83\x11e\xcc\xd8c$\x05\xe2\x11\xf9\xa9\x13\x0c'

In [2]:
#A little different form
" ".join(["{:02X}".format(b) for b in nonce])

'F7 4A CD 59 0D BB 66 25 F7 62 12 01 F8 5B 59 F4 33 B8 33 11 65 CC D8 63 24 05 E2 11 F9 A9 13 0C'

We'll also need an encryption key. Let's use this really bad example:

In [3]:
encryption_key = b'ThisIsNotSecure!'
" ".join(["{:02X}".format(b) for b in encryption_key])

'54 68 69 73 49 73 4E 6F 74 53 65 63 75 72 65 21'

In [4]:
# For an AES-128 cipher, we'll need a 16 byte key.
len(encryption_key)

16

A key derivation function (KDF) may be a better approach for a key based on a password.

### System Users
For a proof of concept, a system server may just use names for logins. While this may be what we see as a user, the actual user data is stored in a hash table, or dictionary. Let's make a user dictionary with a fixed length value as the key and different attributes as the values.

In [5]:
# The key is the user number (integer) and the value is 
# the dictionary of all the properties for the user. 
users = {1:{"name":"Jeremy Daily","role":'admin'},
         2:{"name":"Amazing Student","role":'user'},
         2348597:{"name":"Michaelangelo","role":'guest'}}
users

{1: {'name': 'Jeremy Daily', 'role': 'admin'},
 2: {'name': 'Amazing Student', 'role': 'user'},
 2348597: {'name': 'Michaelangelo', 'role': 'guest'}}

In [6]:
import struct

In [7]:
# For the protocol, we'll also need a client ID, but it needs to be in a 
# fixed width, so we'll pack the user ID in to a fixed width.
client_id = 1
#but this needs to be in bytes:
client_id_bytes = struct.pack(">L",client_id)
" ".join(["{:02X}".format(b) for b in client_id_bytes])

'00 00 00 01'

In [8]:
# The data to encrypt are:
data_to_encrypt = client_id_bytes + nonce
data_to_encrypt

b'\x00\x00\x00\x01\xf7J\xcdY\r\xbbf%\xf7b\x12\x01\xf8[Y\xf43\xb83\x11e\xcc\xd8c$\x05\xe2\x11\xf9\xa9\x13\x0c'

### Symmetric Encryption
Symmetric encryption is usually implemented as a block cipher with some feedback mode to eliminate patterns. 

We'll start with an example of common implementations available as libaries in Python, starting with the crypto library at
https://cryptography.io/.
This library is maintained and provides access to many features and algorithms needed to work with protocols using modern cryptography.

DISCLAIMER: Currently cryptography.io only supports NIST curves, none of which are considered “safe” by the SafeCurves (https://safecurves.cr.yp.to/) project run by Daniel J. Bernstein and Tanja Lange.

If the libraries don't load, you can install them by typing `pip install --upgrade cryptography` at the command prompt (not Jupyter or Python). This notebook was originally written for version 3.1 of the cryptography library. See the changelog: https://cryptography.io/en/latest/changelog/.

In [9]:
# If you don't have the latest version of the cryptography library installed, 
# you can use it to install in the version that is serving the jupyter notebook
# This method may be necessary if you have multiple versions of Python on your computer. 
import sys
#Run this if you are connected to the Internet
!{sys.executable} -m pip install --upgrade --user cryptography
# You may need to restart the Kernel if the requirements are not already up-to-date


[notice] A new release of pip available: 22.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


The following is a brief intro to Symmetric Ciphers where each principal as access to the same secret key. 

https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/?highlight=ecb%20mode#module-cryptography.hazmat.primitives.ciphers

## Advanced Encryption Standard
Unless there is a really compelling reason, you should pick AES as the symmetric cipher. According to Ross Anderson in Section 5.4.2 (third edition):

```The NSA has since 2005 approved AES with 128-bit keys for protecting information up to SECRET and with 192-bit or 256-bit keys for TOPSECRET. So I recommend that you use AES instead of GOST, or Camellia,or even Serpent.  The definitive specification of AES is Federal Information Processing Standard 197, and its inventors have written a book describing its design in detail.``` 

Let's look at how to use AES with cryptography.io.

![logo](cybertruckchallenge.bmp)

In [10]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

The ECB mode is not a secure mode, but it is the simplest.
https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/?highlight=ecb%20mode#cryptography.hazmat.primitives.ciphers.modes.ECB

In [11]:
key_size = 16
key = urandom(key_size)

### Bitmap image encryption
We want to send a scrambled picture through the Internet. Let's keep the original image file, but encrypt the contents. Here's a blurb about how bmp files are constructed.
https://medium.com/sysf/bits-to-bitmaps-a-simple-walkthrough-of-bmp-image-format-765dc6857393
Based on this description, we will keep the original 54 bytes. 

By the way, embedding codes and cryptographic information in pictures is something called steganography. This isn't steganography, but you can learn more about it here: https://www.comptia.org/blog/what-is-steganography

In [12]:
# BMP Images use 54 bytes up front to define the file
offset = 54
#Let's use a graphic as an example 
with open('cybertruckchallenge.bmp','rb') as raw_file:
    logo_bytes = raw_file.read()
data_blocks,extra = divmod(len(logo_bytes)-offset,key_size)
data_blocks

65536

In [13]:
# This is an example of a bad mode for the cipher. NEVER USE THIS!
cipher = Cipher(algorithms.AES(encryption_key), modes.ECB())
cipher

In [14]:
encryptor = cipher.encryptor()

#Start the first few bytes the same as the others
encrypted_picture_bytes = logo_bytes[:offset]

#loop through bytes and ECB encrypt them
encrypted_picture_bytes += encryptor.update(logo_bytes[offset:offset+data_blocks*key_size]) + encryptor.finalize()

In [15]:
encrypted_picture_bytes[:120]

b'BM6\x00\x10\x00\x00\x00\x00\x006\x00\x00\x00(\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x01\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\xc4\x0e\x00\x00\xc4\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@<\xb2\xf6\xa3\xdc.\xdc\x8c@oFwj(\xa8@<\xb2\xf6\xa3\xdc.\xdc\x8c@oFwj(\xa8@<\xb2\xf6\xa3\xdc.\xdc\x8c@oFwj(\xa8@<\xb2\xf6\xa3\xdc.\xdc\x8c@oFwj(\xa8@<'

In [16]:
with open('encrypted_logo_ECB.bmp','wb') as out:
    out.write(encrypted_picture_bytes)

![encrypted_logo](encrypted_logo_ECB.bmp)

### AES-CBC
Let's use a better mode.

In [17]:
iv = urandom(key_size)
cipher = Cipher(algorithms.AES(encryption_key), modes.CBC(iv))
encryptor = cipher.encryptor()

#Start the first few bytes the same as the others
encrypted_picture_bytes = logo_bytes[:offset]

#loop through bytes and ECB encrypt them
encrypted_picture_bytes += encryptor.update(logo_bytes[offset:offset+data_blocks*key_size]) + encryptor.finalize()

with open('encrypted_logo_CBC.bmp','wb') as out:
    out.write(encrypted_picture_bytes)

![encrypted_logo](encrypted_logo_CBC.bmp)


### Bad modes
The ECB mode is nicknamed "electronic coloring book" because it still reveals patterns and is not secure. Other modes should be used instead.

### No integrity checks
The process of encrypting and decrypting data does nothing to detect data alteration. Therefore, an authenticity check is critical. Let's see this through an example of encrypting the constitution preamble. First, let's encrypt the preamble with the Fernet recipe that includes a timestamp, an acceptable mode, and a message authentication code.

In [18]:
# Let's start with some plain text:
plain_text = "We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America."
print(plain_text)


We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.


In [19]:
# Out of the box recipe for using fast symmetric encryption
from cryptography.fernet import Fernet

In [20]:
# Generate a key suitable for using the Fernet protocol
shared_key = Fernet.generate_key()

In [21]:
# The key should be kept safe (This is the hard engineering problem)
print(shared_key)

b'O_IbfXWAegAiJXAv2O34Jla_FDiJVDV4ymr_lrRPxu8='


In [22]:
len(shared_key)

44

The key generated is base64 encoded using a URL safe library. 

https://tools.ietf.org/html/rfc3548.html decribes the url and filesystem safe encoding.

As described in 
https://www.pythoninformer.com/python-libraries/cryptography/fernet/ or https://www.comparitech.com/blog/information-security/what-is-fernet/,
the generated key is contains two 16-byte randomly generated keys that are concatenated and base64 encoded.

In [23]:
import base64

In [24]:
#This is actually two 16-byte keys concatenated and  32-byte (256 bit) base-64 random key
key_bytes = base64.urlsafe_b64decode(shared_key)
print(key_bytes)

b';\xf2\x1b}u\x80z\x00"%p/\xd8\xed\xf8&V\xbf\x148\x89T5x\xcaj\xff\x96\xb4O\xc6\xef'


In [25]:
# There should be 32-bytes of key material
len(key_bytes)

32

In [26]:
# The first 16 bytes (128 bits) is the signing key
signing_key = key_bytes[:16]
signing_key

b';\xf2\x1b}u\x80z\x00"%p/\xd8\xed\xf8&'

In [27]:
# The second half of the key is the encryption key
encryption_key = key_bytes[16:]
encryption_key

b'V\xbf\x148\x89T5x\xcaj\xff\x96\xb4O\xc6\xef'

In [28]:
# create an instance of the Fernet cipher tool
f = Fernet(shared_key)
print(f)

Fernet methods are documented at https://cryptography.io/en/latest/fernet/

In [29]:
#Modern ciphers only work on numbers, so we need to convert plaintext to bytes
plain_bytes = plain_text.encode('utf-8')
plain_bytes

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.'

In [30]:
#the result is ready to be sent over the internet or stored on a machine
cipher_token = f.encrypt(plain_bytes)
print(cipher_token) # This is what can be sent across the Internet

b'gAAAAABkG4jf6fJOouoFv4RNeSwnziXJ5z6Q4DPHtMvR_NcBQ7SFdtVdzp2iRXJofNq98yakAY3rheGQyKMlM4J7vltuSa7znLrkNMGtdAKC9-Bv1G7PpcpCIP4zbqd4A_Z5Sg5CtoTDo1EwmmpB_UDKiBIVOgN6tPutuPNrkEd_0vERf9x4YgRvmO00fhTs0wjudjqVbJcu4SoLUzaAgah482yHTCDFOBdaJ2jo6Cb75FTqkdkhPmUtWSfQA-gnKKvrFKSZ3dZG3YcIJ9TtK1w0GuFOo3dlyNZGRToFuWq9rVlHYrM1cl2uPDcCEV4j6wySiXp3p8Pq-LtmpOLpubBGMICM3jZXJgmKqVR4S5v6seo6YtaYY5d15g_q7nbMyJs3Y1NrJVqhCtdsB2T-iZswL8jDQdm8CLeAw2upGtKfamgS-nigXl_O_jhhUtI9UHLyKqhOPscMT810qfMl6nmn65qcXKmyomEOg1LSjmb9IotqliWBakJ1ePYvNfSIXWl9vJhGiWLD'


In [31]:
#The inverse is to decrypt to get bytes
f.decrypt(cipher_token)

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.'

From https://www.pythoninformer.com/python-libraries/cryptography/fernet/,
The Fernet protocol implements a more complete symmetric cipher system by

* Providing a secure mechanism for generating keys (a key is similar to a password).
* Selection a secure encryption algorithm (AES using CBS mode and PKCS7 padding)
* Randomly allocating a secure initialization vector (IV) or "salt" to make the encryption more secure.
* Timestamping the encrypted message.
* Signing the message (using HMAC and SHA256) to detect any attempts to change it.

Let's disect the cipher token to see these individual attributes.

### Keys from passwords
Let's generate a valid Fernet key with a passcode. The passcode could be shared verbally instead of electronically.
We'll need to use some primitives. Reference: https://cryptography.io/en/latest/fernet/#using-passwords-with-fernet


We also have to use good random numbers for the initialization vectors.
https://docs.python.org/3/library/secrets.html

In [32]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

PBKDF2 = Password Based Key Derivation Function 2

https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/#pbkdf2

In [33]:
# We will start with a desired pass phrase as bytes
passcode = b"I'm a passphrase"
passcode

b"I'm a passphrase"

In [34]:
# generate 16 random bytes for the initialization vector or salt
iv = urandom(16)
iv

b'\xb3\xbd\x97\x91f\xaa\x0fY\x1e\xd1\x00\x87\x1a\xa0l='

In [35]:
#Key derivation function. This is practically  irreversible 
kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=iv,
        iterations=100000,
    )
kdf_result = kdf.derive(passcode)
print(kdf_result)

b'\x1dDMN\xce\xf1:iFr\x90\x0f>15Yq\x80\xb6\x0c \x05]\xd1\x10\x1f\xafr\nH\x04X'


In [36]:
# We should have 32 bytes now
len(kdf_result)

32

In [37]:
# Make a valid fernet key:
fernet_key = base64.urlsafe_b64encode(kdf_result)
fernet_key

b'HURNTs7xOmlGcpAPPjE1WXGAtgwgBV3REB-vcgpIBFg='

In [38]:
# Check to see if you can create a Fernet cipher
f1 = Fernet(fernet_key)
print(f1)

If both parties know the passphrase and salt value, then each party could derive the same key and send encrypted messages.

## Encrypted Data
Let's see how the data is packed and encrypted. 
The enciphered token has the following structure:

1. Version [1 byte] - the only valid value currently is 128.
2. Timestamp [8 bytes] - a 64 bit, unsigned, big-endian integer that indicates when the ciphertext was created. 
3. IV [16 bytes] - the 128 bit Initialization Vector used in AES encryption and decryption.
4. Ciphertext [16 * N Bytes, where N is the number of cipher blocks] - the encrypted version of the plaintext message. This is encrypted using AES128 in CBC mode using the PKCS7 padding algorithm.
5. HMAC [32 bytes] - a 256-bit hash based message authentication code of the concatenated Version, Timestamp, IV, and Ciphertext fields. The HMAC is signed using the signing key section of the Fernet key.

In [39]:
# Recall the base64 encoded cipher token
cipher_token

b'gAAAAABkG4jf6fJOouoFv4RNeSwnziXJ5z6Q4DPHtMvR_NcBQ7SFdtVdzp2iRXJofNq98yakAY3rheGQyKMlM4J7vltuSa7znLrkNMGtdAKC9-Bv1G7PpcpCIP4zbqd4A_Z5Sg5CtoTDo1EwmmpB_UDKiBIVOgN6tPutuPNrkEd_0vERf9x4YgRvmO00fhTs0wjudjqVbJcu4SoLUzaAgah482yHTCDFOBdaJ2jo6Cb75FTqkdkhPmUtWSfQA-gnKKvrFKSZ3dZG3YcIJ9TtK1w0GuFOo3dlyNZGRToFuWq9rVlHYrM1cl2uPDcCEV4j6wySiXp3p8Pq-LtmpOLpubBGMICM3jZXJgmKqVR4S5v6seo6YtaYY5d15g_q7nbMyJs3Y1NrJVqhCtdsB2T-iZswL8jDQdm8CLeAw2upGtKfamgS-nigXl_O_jhhUtI9UHLyKqhOPscMT810qfMl6nmn65qcXKmyomEOg1LSjmb9IotqliWBakJ1ePYvNfSIXWl9vJhGiWLD'

In [40]:
cipher_token_bytes = base64.urlsafe_b64decode(cipher_token)
cipher_token_bytes

b'\x80\x00\x00\x00\x00d\x1b\x88\xdf\xe9\xf2N\xa2\xea\x05\xbf\x84My,\'\xce%\xc9\xe7>\x90\xe03\xc7\xb4\xcb\xd1\xfc\xd7\x01C\xb4\x85v\xd5]\xce\x9d\xa2Erh|\xda\xbd\xf3&\xa4\x01\x8d\xeb\x85\xe1\x90\xc8\xa3%3\x82{\xbe[nI\xae\xf3\x9c\xba\xe44\xc1\xadt\x02\x82\xf7\xe0o\xd4n\xcf\xa5\xcaB \xfe3n\xa7x\x03\xf6yJ\x0eB\xb6\x84\xc3\xa3Q0\x9ajA\xfd@\xca\x88\x12\x15:\x03z\xb4\xfb\xad\xb8\xf3k\x90G\x7f\xd2\xf1\x11\x7f\xdcxb\x04o\x98\xed4~\x14\xec\xd3\x08\xeev:\x95l\x97.\xe1*\x0bS6\x80\x81\xa8x\xf3l\x87L \xc58\x17Z\'h\xe8\xe8&\xfb\xe4T\xea\x91\xd9!>e-Y\'\xd0\x03\xe8\'(\xab\xeb\x14\xa4\x99\xdd\xd6F\xdd\x87\x08\'\xd4\xed+\\4\x1a\xe1N\xa3we\xc8\xd6FE:\x05\xb9j\xbd\xadYGb\xb35r]\xae<7\x02\x11^#\xeb\x0c\x92\x89zw\xa7\xc3\xea\xf8\xbbf\xa4\xe2\xe9\xb9\xb0F0\x80\x8c\xde6W&\t\x8a\xa9TxK\x9b\xfa\xb1\xea:b\xd6\x98c\x97u\xe6\x0f\xea\xeev\xcc\xc8\x9b7cSk%Z\xa1\n\xd7l\x07d\xfe\x89\x9b0/\xc8\xc3A\xd9\xbc\x08\xb7\x80\xc3k\xa9\x1a\xd2\x9fjh\x12\xfax\xa0^_\xce\xfe8aR\xd2=Pr\xf2*\xa8N>\xc7\x0cO\xcdt\xa9\xf3%\xeay\xa7\xeb\x

In [41]:
#parse the fields
version = cipher_token_bytes[0]
print("Version: {}".format(version))
time_stamp = cipher_token_bytes[1:9]
print("time_stamp: {}".format(time_stamp))
init_vect = cipher_token_bytes[9:25]
print("init_vect: {}".format(init_vect))
cipher_data = cipher_token_bytes[25:-32]
print("Length of cipher_data: {}".format(len(cipher_data)))
hmac_value = cipher_token_bytes[-32:]
print("hmac_value: {}".format(hmac_value))

Version: 128
time_stamp: b'\x00\x00\x00\x00d\x1b\x88\xdf'
init_vect: b"\xe9\xf2N\xa2\xea\x05\xbf\x84My,'\xce%\xc9\xe7"
Length of cipher_data: 336
hmac_value: b'a\x0e\x83R\xd2\x8ef\xfd"\x8bj\x96%\x81jBux\xf6/5\xf4\x88]i}\xbc\x98F\x89b\xc3'


In [42]:
#Convert the timestamp bytes into an integer
#This is the number of seconds from the Jan 1, 1970 epoch
import struct
time_integer = struct.unpack(">Q",time_stamp)[0]
time_integer

1679526111

In [43]:
#convert the timestamp into iso format so we can read it.
from datetime import datetime
datetime.fromtimestamp(time_integer).isoformat()

'2023-03-22T17:01:51'

For more on time operations in Python: https://docs.python.org/3/library/datetime.html

In [44]:
#Recall (no time constraints)
f.decrypt(cipher_token)

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.'

In [67]:
# If more than ttl seconds have passed from the encryption, then the token is bad. 
# This is helpful to guarantee the freshness of data coming from a real-time producer.
# (i.e. instrument data from a test vehicle streaming in real-time. )
f.decrypt(cipher_token,ttl = 10)

InvalidToken: 

### AES-CBC decryption
We have the keys used for the symmetric encryption along with the IV, so we can create an AES cipher based on these inputs and decrypt the cipher text. We'll need to import these primitives. Reference: https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/

In [46]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [47]:
#Recall
encryption_key

b'V\xbf\x148\x89T5x\xcaj\xff\x96\xb4O\xc6\xef'

In [48]:
cipher = Cipher(algorithms.AES(encryption_key), modes.CBC(init_vect))
cipher

In [49]:
decryptor = cipher.decryptor()
decryptor

In [50]:
#Notice the padding at the end
# if this gives an error, reinitialize the decryptor
clear = decryptor.update(cipher_data) + decryptor.finalize()
clear

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.\t\t\t\t\t\t\t\t\t'

In [51]:
# The padding is performed by inserting N bytes at the end of the clear bytes
# The pad length N is encoded as the pad value. The pad value is determined as chr(N).
# The inverse of chr(N) is ord(char)
ord('\t')

9

There were 9 b'/t' characters padded to the end of the preamble to make the block divisible by 16.

In [52]:
from cryptography.hazmat.primitives import padding

In [53]:
# Let's use the primitive (16 bytes = 128 bits)
unpadder = padding.PKCS7(128).unpadder()
unpadded_clear = unpadder.update(clear)
unpadded_clear

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of A'

In [54]:
#We are still missing the last part:
unpadder.finalize()

b'merica.'

In [55]:
#Putting the two together
unpadder = padding.PKCS7(128).unpadder()
unpadded_clear = unpadder.update(clear) + unpadder.finalize()
unpadded_clear

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.'

### Tamper Detection
What if someone manipulates a bit in the cipher_data? The decryption will still work, but the message will be changed. We need to detect these manipulations before presenting actual data. The decryptor does not know if the data bits were rearranged.

Key Point: encryption does not mean authentication. 

In [56]:
#Change to a bytearray to manipulate
altered_cipher = bytearray(cipher_data)
altered_cipher == bytearray(cipher_data)

True

In [57]:
#Change a byte
altered_cipher[29] = 0
altered_cipher == bytearray(cipher_data)

False

In [58]:
# use the same decryptor and decrypt
decryptor = cipher.decryptor()
decryptor.update(cipher_data) + decryptor.finalize()

b'We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.\t\t\t\t\t\t\t\t\t'

In [59]:
# in cipher block chaining (CBC) mode, some of the original text will come through after manipulating a byte.
decryptor = cipher.decryptor()
decryptor.update(altered_cipher) + decryptor.finalize()

b'We the People ofbmYA4\xe45\xff\x85\xfd\\qqx\xc1\x84es, in Order uo form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America.\t\t\t\t\t\t\t\t\t'

How can someone tell if the enciphered data was manipulated?

Ans: use a message authentication code.

https://cryptography.io/en/latest/hazmat/primitives/mac/hmac/

In [60]:
from cryptography.hazmat.primitives import hashes, hmac

In [61]:
#recall
signing_key

b';\xf2\x1b}u\x80z\x00"%p/\xd8\xed\xf8&'

In [62]:
#Add the concatenated message bytes to the HMAC and produce the digest
h = hmac.HMAC(signing_key,hashes.SHA256())
h.update(cipher_token_bytes[:-32])
new_hmac_digest = h.finalize()
new_hmac_digest

b'a\x0e\x83R\xd2\x8ef\xfd"\x8bj\x96%\x81jBux\xf6/5\xf4\x88]i}\xbc\x98F\x89b\xc3'

In [63]:
#Compare
hmac_value == new_hmac_digest

True

In [64]:
# Using verify
h = hmac.HMAC(signing_key,hashes.SHA256())
h.update(cipher_token_bytes[:-32])
h.verify(cipher_token_bytes[-32:])
# If the verify command doesn't raise an error, it's ok.
print("Message Authenticated")

Message Authenticated


In [65]:
# Using verify
h = hmac.HMAC(signing_key,hashes.SHA256())
# Remove the first byte
h.update(cipher_token_bytes[1:-32])
h.verify(cipher_token_bytes[-32:])

InvalidSignature: Signature did not match digest.

In [66]:
# Use a try -except block to catch the verification error
from cryptography import exceptions
h = hmac.HMAC(signing_key,hashes.SHA256())
# Remove the first byte
h.update(cipher_token_bytes[1:-32])
try:
    h.verify(cipher_token_bytes[-32:])
    print("Message Authenticated")
except exceptions.InvalidSignature:
    print("Invalid Signature")

Invalid Signature


## Section Summary
In this section we disected the Fernet protocol for symmetric encryption. In this process we discussed
* Improper Modes of Encryption 
* Key Generation
* Keys from Passwords
* Availability Controls with Timestamps
* Confidentiality Controls using AES in the cipher block chaining mode
* Integrity Controls and Tamper Detection using HMAC
The Fernet recipe is a good starting point to send data across the Internet between trusted parties with a shared key.